<a href="https://colab.research.google.com/github/ssosoo/2024_DS60/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D/%EB%8B%A4%EC%B8%B5%ED%8D%BC%EC%85%89%ED%8A%B8%EB%A1%A0_%EC%86%90%EA%B8%80%EC%94%A8%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

손글씨 실습을 위해 MNIST에서 제공하는 데이터셋 호출

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


데이터는 흑백 데이터다. 0에 가까울수록 흰색, 255에 가까울수록 검은색 픽셀을 의미한다.

In [4]:
#각 픽셀(28x28) 해당값 확인
#첫번째 이미지의 8번째 행 값
print(x_train[0][8])

[  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241
   0   0   0   0   0   0   0   0   0   0]


In [5]:
# 8번째 행의 8번째 픽셀 값
print(x_train[0][8][8])

219


In [6]:
#각 데이터에 해당하는 레이블 숫자 (0~9) 확인
#mnist 데이터에서 y레이블은 실제 숫자를 의미한다
print(y_train[0:9])


[5 0 4 1 9 2 1 3 1]


학습 효율을 위해 데이터 정규화를 진행한다.

MNIST의 모든 값은 0~255 사이이므로
 모든 값을 255로 나누어 0부터 1사이로 정규화한다.

In [7]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_test /= gray_scale

In [10]:
model = Sequential([
    Flatten(input_shape = (28,28)), # 차원 축소
    Dense(256, activation='relu'),  # 첫 번째 히든 레이어 (h1)
    Dense(128, activation='relu'),  # 두 번째 히든 레이어 (h2)
    Dropout(0.1),                   # 두 번째 히든 레이어에 Dropout 10% 적용
    Dense(10),                      # 세 번째 히든 레이어 (logit)
    Activation('softmax')           # softmax layer
    ])

In [11]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_2 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 256)                 │         200,960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 10)                  │           1,290 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 10)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 235,146 (918.54 KB)

 Trainable params: 235,146 (918.54 KB)

 Non-trainable params: 0 (0.00 B)

첫 번째 레이어에 784개의 입력을 받는 256개의 노드가 존재하고, 노드마다 편향값이 하나씩 존재하므로 784*256(weight) + 256(bias) = 200960의 파라미터가 존재한다.

In [12]:
# 손실함수와 최적화 방법을 모델에 적용한다.

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy', # 레이블을 원 핫 인코딩으로 자동으로 변경해 크로스 엔트로피를 측정
              metrics=['accuracy'])

In [16]:
# 매 주기마다 검증 정확도를 측정합니다.
# 검증 정확도가 5번 연속으로 개선되지 않을 경우, 조기 종료를 수행합니다.
# 최종 저장 모델은 검증 정확도가 가장 높은 모델입니다.

callbacks = [EarlyStopping(monitor = 'val_accuracy', patience = 5, restore_best_weights = False),
             ModelCheckpoint(filepath = 'best_model.keras', monitor = 'val_accuracy',
                             save_best_only = True)]

In [17]:
# 학습 진행
model.fit(x_train, y_train, epochs=300, batch_size=1000, validation_split=0.1,
          callbacks=callbacks)

Epoch 1/300
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.6671 - loss: 1.1592 - val_accuracy: 0.9393 - val_loss: 0.2163
Epoch 2/300
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.9258 - loss: 0.2541 - val_accuracy: 0.9600 - val_loss: 0.1481
Epoch 3/300
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.9464 - loss: 0.1834 - val_accuracy: 0.9663 - val_loss: 0.1199
Epoch 4/300
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.9591 - loss: 0.1371 - val_accuracy: 0.9727 - val_loss: 0.1004
Epoch 5/300
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.9687 - loss: 0.1093 - val_accuracy: 0.9722 - val_loss: 0.0925
Epoch 6/300
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.9737 - loss: 0.0895 - val_accuracy: 0.9762 - val_loss: 0.0804
Epoch 7/300
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.9788 - loss: 0.0720 - val_accuracy: 0.9792 - val_loss: 0.0751
Epoch 8/300
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.9829 - loss: 0.0615 - val_accuracy: 0.

In [18]:
# 검증 정확도가 가장 높은 모델을 대상으로 테스트 진행
results = model.evaluate(x_test, y_test, verbose = 0)
print('test loss, test acc:', results)

test loss, test acc: [0.06853991746902466, 0.9811999797821045]
